In [1]:
import pandas as pd
import numpy as np

#load the dataset into a pandas dataframe
df = pd.read_csv("../Movies/Data/movies_metadata.csv")

#Display the first five movies in the dataframe
df.head()

##### > Guidlines
##### 1. choose a metric(or score) to rate the movies on
##### 2. Decide on the prerequisites for the movies to be featured on the chart
##### 3. Calculate the score for every movie that satisfies the condition
##### 4. output the list of movies in decreasing order of their score

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### Building the simple recommender steps
##### 1. choose a metric(or score) to rate the movies on

In [13]:
# we shall using IMDB's weighted rating formula as our metric

# Weighted Rating (WR) = ((v/v+m)*R) + ((m/v+m)*C)

# v = no. of votes garnered by the movie
# m = min. number of votes required for a movie to be in the chart(prerequisite)
# R = mean rating of the movie
# C = mean rating of all the movies in the dataset

# n/b we already have v & R for every movie i form of "vote_count" and "vote_average"

In [14]:
# calculate the number of votes garnered by the 80th percentile movie
m = df['vote_count'].quantile(0.80)
m

50.0

In [20]:
# only consider movies longer than 45 min and shorter than 300 min
q_movies = df[(df['runtime'] >= 45) & (df['runtime'] <=300)]

# only consider movies that have garnered more than m votes
q_movies = q_movies[q_movies['vote_count'] >=m]

# inspect the number of movies that made the cut
q_movies.shape

(8963, 24)

##### Calculating the score

In [23]:
#calculating C

c = df['vote_average'].mean()
c


5.618207215133889

In [28]:
# Function to compute the IMDB weighted rating for each movie

def weighted_rating(x, m=m, c=c):
    v = x['vote_count']
    R = x['vote_average']
    
    # compute the weighted score
    return ((v/v+m)*R) + (m/(m+v) * c)


In [35]:
# compute the score using the weighted rating function defined above
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [34]:
#Print the top 25 movies
q_movies[['title', 'vote_count', 'vote_average', 'score', 'runtime']].head(25)

,title,vote_count,vote_average,score,runtime
43320,Planet Earth II,50.0,9.5,487.309104,300.0
10309,Dilwale Dulhania Le Jayenge,661.0,9.1,464.495092,190.0
7643,Sansho the Bailiff,68.0,8.7,446.080596,124.0
29183,The Jinx: The Life and Deaths of Robert Durst,85.0,8.6,440.680817,240.0
33420,Human,98.0,8.6,440.498043,263.0
40251,Your Name.,1030.0,8.5,433.760102,106.0
834,The Godfather,6024.0,8.5,433.546248,175.0
314,The Shawshank Redemption,8358.0,8.5,433.533410,142.0
39094,Bo Burnham: Make Happy,56.0,8.4,431.050098,60.0
24990,Doctor Who: The Time of the Doctor,61.0,8.4,430.930724,60.0
